# WebScraping Assignment - 3

In [1]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver
from selenium.webdriver.common.by import By

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

# importing regex
import re

import warnings
warnings.filterwarnings('ignore')
import time

# Q1- Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)


# Opening the homepage of Amazon.in
url = "https://www.amazon.in/"
driver.get(url)
time.sleep(2)


# Asking the user to input the keywords he/she wants to search
user_inp = input('Enter the product you want to search : ')

Enter the product you want to search : guitar


In [3]:
search_bar = driver.find_element(By.ID,"twotabsearchtextbox")    # Locating searc_bar by id
search_bar.clear()                                               # clearing search_bar
search_bar.send_keys(user_inp)                                   # sending user input to search bar
search_button = driver.find_element(By.XPATH,'//div[@class="nav-search-submit nav-sprite"]/span/input')       # Locating search_button by xpath
search_button.click()                                                                # Clicking the button to start search

# Q2- In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [4]:
# scrape all product urls
product_urls = []
start=0
end=3
for page in range(start,end):                                                 #for loop for scrapping 3 page
    url=driver.find_elements(By.XPATH,'//a[@class = "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href"))                        #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')     #scraping the list of buttons from the page
    nxt_button.click()
    time.sleep(2)

In [5]:
len(product_urls)

179

In [10]:
# create blank lists  
product = []
price = []
availability=[]
Brand=[]
ReturnExchange=[]
Delivery=[]
Ratings=[]


for url in product_urls[:10]:                                                        ##loop for every guitar in the list
    driver.get(url)
    time.sleep(2)
    
    try:
        brand= driver.find_element(By.XPATH,'//*[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]/span')
        Brand.append(brand.text)
    except NoSuchElementException:
        Brand.append('-') 
        
    try:
        exchange = driver.find_element(By.XPATH,'//*[@id="RETURNS_POLICY"]/span/div[2]/a')
        ReturnExchange.append(exchange.text)
    except NoSuchElementException:
        ReturnExchange.append('-') 
        
    try:
        delivery= driver.find_element(By.XPATH,'//span[@class="a-text-bold"]')
        Delivery.append(delivery.text)
    except NoSuchElementException:
        Delivery.append('-') 
        
    try:
        product.append(driver.find_element(By.XPATH,'//span[@id="productTitle"]').text)
    except:
        product.append('-')
    
    try:
        
        try:
            price.append(driver.find_element(By.XPATH,'//td[@class="a-span12"]').text)
        except:
            price.append(driver.find_element(By.CSS_SELECTOR,'span[class="a-price-whole"]').text)             
    except:
        price.append('-')
    
    try:
        availability.append(driver.find_element(By.XPATH,'//div[@id="availability"]').text)
    except:
        availability.append('-')
        
    try:
        Ratings.append(driver.find_element(By.XPATH,'//span[@class="a-size-medium"]').text)
    except:
        Ratings.append('-')
         

In [12]:
print(len(Brand),len(ReturnExchange),len(Delivery),len(product),len(price),len(availability),len(Ratings))

10 10 10 10 10 10 10


In [13]:
df=pd.DataFrame({'Brand':Brand,'ReturnExchange':ReturnExchange,'Price':price,'Delivery_by':Delivery,'urls':url,'Ratings':Ratings,'Name of Product':product,'Availability of product':availability})
df

,Brand,ReturnExchange,Price,Delivery_by,urls,Ratings,Name of Product,Availability of product
0,Kadence,7 Days Replacement,"₹4,899.00","Sunday, 4 September",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,,"Kadence Frontier Series, Acoustic Guitar With/...",In stock.
1,Kadence,7 Days Replacement,"₹6,799.00","Sunday, 4 September",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,In stock.
2,Medellin,7 Days Replacement,"₹2,499.00","Sunday, 4 September",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,,"Medellin 38"" Acoustic Guitar Blue Burst Carbon...",In stock.
3,FESTRA,7 Days Replacement,"2,695","Tuesday, 6 September",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,-,"Festra 38C, 38 Inch Acoustic Guitar Kit with B...",In stock.
4,Juârez,7 Days Replacement,"₹2,090.00","Sunday, 4 September",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",In stock.
5,Belear,7 Days Replacement,"2,999","Tuesday, 6 September",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,-,Belear I-280 38 Inch Blue Spruce Cutaway Acous...,In stock.
6,Intern,7 Days Replacement,"₹2,190.00","Friday, 9 September",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,,"Intern INT-38C Acoustic Guitar Kit, With Bag, ...",
7,Kadence,7 Days Replacement,"₹4,899.00","Sunday, 4 September",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,,"Kadence Frontier Series, Acoustic Guitar With/...",In stock.
8,Kadence,7 Days Replacement,"₹6,799.00","Sunday, 4 September",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,In stock.
9,YAMAHA,7 Days Replacement,"10,250",,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,-,"yamaha acoustic guitar F310, 6-Strings Acousti...",Only 1 left in stock.


In [14]:
driver.close()

# Q3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [15]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)


# Opening the google images
url = "https://images.google.com/"
driver.get(url)
search_bar = driver.find_element(By.XPATH,'//input[@class="gLFyf gsfi"]')    # Finding the search bar using it's xpath
search_bar.send_keys("fruits")                                               # Inputing "fruits" keyword to search images

In [16]:
search_button = driver.find_element(By.XPATH,'//div[@class="zgAlFc"]')    # Finding the xpath of search button
search_button.click()                                                     # Clicking the search button

In [27]:
# 500 times we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\Sunny\Downloads"+str(i)+".jpg", "wb")
    file.write(response.content)

In [28]:
driver.close()

In [29]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)


# Opening the google images
url = "https://images.google.com/"
driver.get(url)
search_bar = driver.find_element(By.XPATH,'//input[@class="gLFyf gsfi"]')    # Finding the search bar using it's xpath
search_bar.send_keys("cars")                                                 # Inputing "cars" keyword to search images

In [30]:
search_button = driver.find_element(By.XPATH,'//div[@class="zgAlFc"]')    # Finding the xpath of search button
search_button.click()                                                     # Clicking the search button

In [32]:
# 500 times we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
img = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in img:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\Sunny\Downloads"+str(i)+".jpg", "wb")
    file.write(response.content)

In [33]:
driver.close()

In [35]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)


# Opening the google images
url = "https://images.google.com/"
driver.get(url)
search_bar = driver.find_element(By.XPATH,'//input[@class="gLFyf gsfi"]')    # Finding the search bar using it's xpath
search_bar.send_keys("cakes")                                                 # Inputing "cakes" keyword to search images

search_button = driver.find_element(By.XPATH,'//div[@class="zgAlFc"]')    # Finding the xpath of search button
search_button.click()                                                     # Clicking the search button

In [36]:
# 500 times we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
cakes = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in cakes:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\Sunny\Downloads"+str(i)+".jpg", "wb")
    file.write(response.content)

In [37]:
driver.close()

In [38]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)


# Opening the google images
url = "https://images.google.com/"
driver.get(url)
search_bar = driver.find_element(By.XPATH,'//input[@class="gLFyf gsfi"]')    # Finding the search bar using it's xpath
search_bar.send_keys("guitar")                                               # Inputing "guitar" keyword to search images

search_button = driver.find_element(By.XPATH,'//div[@class="zgAlFc"]')    # Finding the xpath of search button
search_button.click()                                                     # Clicking the search button

In [39]:
# 500 times we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
guitar = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in guitar:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\Sunny\Downloads"+str(i)+".jpg", "wb")
    file.write(response.content)

In [40]:
driver.close()

In [41]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)


# Opening the google images
url = "https://images.google.com/"
driver.get(url)
search_bar = driver.find_element(By.XPATH,'//input[@class="gLFyf gsfi"]')    # Finding the search bar using it's xpath
search_bar.send_keys("machine learning")                                     # Inputing "machine learning" keyword to search images

search_button = driver.find_element(By.XPATH,'//div[@class="zgAlFc"]')    # Finding the xpath of search button
search_button.click()                                                     # Clicking the search button

In [43]:
# 500 times we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
ML = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in ML:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\Sunny\Downloads"+str(i)+".jpg", "wb")
    file.write(response.content)

In [44]:
driver.close()

# Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [76]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)

# Opening the homepage of flipkart.com
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(2)

In [77]:
try:
    login_X_button = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _2doB4z"]')           # Button to close login popup
    login_X_button.click()
except NoSuchElementException : 
    print("No Login page")

In [78]:
# Finding the search bar using it's xpath
search_bar = driver.find_element(By.XPATH,'//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')    
search_bar.clear()                                                           # Clearing the search bar
search_bar.send_keys("pixel")                                              # Inputing keyword to search
search_button = driver.find_element(By.XPATH,'//button[@class="L0Z3Pu"]')    # Finding the xpath of search button
search_button.click()                                                        # Clicking the search button

In [79]:
# Fetching urls of phones coming on 1st page
flip_urls = []
urls = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for url in urls:
    flip_urls.append(url.get_attribute("href"))
   
#Make empty lists   
flip_dict = {}
flip_dict["Brand"] = []
flip_dict["Smartphone"] = []
flip_dict["Colour"] = []
flip_dict["RAM"] = []
flip_dict["Storage(ROM)"] = []
flip_dict["Primary Camera"] = []
flip_dict["Secondary Camera"] = []
flip_dict["Display Size"] = []
flip_dict["Battery Capacity"] = []
flip_dict["Price"] = []
flip_dict["URL"] = []

In [80]:
# Scraping data from each url
for url in flip_urls:
    driver.get(url)    # Saving url                                                     
    print("Scraping URL = ", url)
    flip_dict['URL'].append(url)                                                          # Loading the webpage by url
    time.sleep(2)
    
    try:
        read_more = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')     # Button for expanding the specs
        read_more.click()
    except NoSuchElementException:
        print("Exception Occured. Moving to next page")
        
    try:
        brand = driver.find_element(By.XPATH,'//span[@class="B_NuCI"]')      # Extracting Brand from xpath
        flip_dict["Brand"].append(brand.text.split()[0])
    except NoSuchElementException:
        flip_dict['Brand'].append('-')
        
    try:
        name = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')      # Extracting Name from xpath
        flip_dict['Smartphone'].append(name.text)
    except NoSuchElementException:
        flip_dict['Smartphone'].append('-')
        
    try:
        color = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      # Extracting colour from xpath
        flip_dict['Colour'].append(color.text)
    except NoSuchElementException:
        flip_dict['Colour'].append('-')
        
    try:
        ram = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                # Extracting RAM from xpath
        flip_dict['RAM'].append(ram.text)
    except NoSuchElementException:
        flip_dict['RAM'].append('-')
        
    try:
        rom = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')         # Extracting Storage(ROM) from xpath
        flip_dict['Storage(ROM)'].append(rom.text)
    except NoSuchElementException:
        flip_dict['Storage(ROM)'].append('-')
        
    try:                                                                                  
        pri_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')     # Extracting Camera from xpath
        flip_dict['Primary Camera'].append(pri_cam.text)
    except NoSuchElementException:
        flip_dict['Primary Camera'].append('-')
        
    try:                                                                                    # Extracting Secondary Camera from xpath
        cam_chk = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if cam_chk != "Secondary Camera" : 
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        flip_dict['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        flip_dict['Secondary Camera'].append('-')
        
    try:
        disp_chk = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  # Extracting Display Size from xpath
        flip_dict['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        flip_dict['Display Size'].append('-')
        
    try:
        if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :   # Extracting Battery Capacity from xpath
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :     
                bat_chk = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
            bat_cap = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Capacity'].append(bat_cap.text)
    except NoSuchElementException:
        flip_dict['Battery Capacity'].append('-')
        
    try:
        price = driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')      # Extracting Price from xpath
        flip_dict['Price'].append(price.text)
    except NoSuchElementException:
        flip_dict['Price'].append('-')

Scraping URL =  https://www.flipkart.com/google-pixel-6a-chalk-128-gb/p/itme5ae89135d44e?pid=MOBGFWEZ5SKU84Z8&lid=LSTMOBGFWEZ5SKU84Z8FXPB45&marketplace=FLIPKART&q=pixel&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=cba187cc-d2fd-49df-96bd-8398f8f27101.MOBGFWEZ5SKU84Z8.SEARCH&ppt=hp&ppn=homepage&ssid=99eag2a5sw0000001662028516927&qH=ab4086ecd47c568d
Scraping URL =  https://www.flipkart.com/google-pixel-6a-charcoal-128-gb/p/itme5ae89135d44e?pid=MOBGFKX5YUXD74Z3&lid=LSTMOBGFKX5YUXD74Z3MXA2OB&marketplace=FLIPKART&q=pixel&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=cba187cc-d2fd-49df-96bd-8398f8f27101.MOBGFKX5YUXD74Z3.SEARCH&ppt=hp&ppn=homepage&ssid=99eag2a5sw0000001662028516927&qH=ab4086ecd47c568d
Scraping URL =  https://www.flipkart.com/google-pixel-4a-just-black-128-gb/p/itm023b9677aa45d?pid=MOBFUSBNAZGY7HQU&lid=LSTMOBFUSBNAZGY7HQUWHTF0C&marketplace=FLIPKART&q=pixel&store=tyy%2F4io&srno=s_1_3&otracker=search&otracker1=search&fm=

In [81]:
print(len(flip_dict["Brand"]),len(flip_dict["Smartphone"]),len(flip_dict["Colour"]),len(flip_dict["Price"]),len(flip_dict['URL']),len(flip_dict["RAM"]),len(flip_dict["Storage(ROM)"]),len(flip_dict["Primary Camera"]),len(flip_dict["Secondary Camera"]),len(flip_dict["Display Size"]),len(flip_dict["Battery Capacity"]))

24 24 24 24 24 24 24 24 24 24 24


In [82]:
flip_df = pd.DataFrame.from_dict(flip_dict)
flip_df

,Brand,Smartphone,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,URL
0,Google,Pixel 6a,Chalk,6 GB,128 GB,12.2MP + 12MP,8MP Front Camera,15.6 cm (6.14 inch),4410 mAh,"₹43,999",https://www.flipkart.com/google-pixel-6a-chalk...
1,Google,Pixel 6a,Charcoal,6 GB,128 GB,12.2MP + 12MP,8MP Front Camera,15.6 cm (6.14 inch),4410 mAh,"₹43,999",https://www.flipkart.com/google-pixel-6a-charc...
2,Google,Pixel 4a,Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),3140 mAh,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
3,Google,Pixel 3a,Clearly White,4 GB,64 GB,12.2MP Rear Camera,8MP Front Camera,14.22 cm (5.6 inch),3000 mAh,"₹39,749",https://www.flipkart.com/google-pixel-3a-clear...
4,Realme,C30,Bamboo Green,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹6,899",https://www.flipkart.com/realme-c30-locked-air...
5,Realme,C30,Lake Blue,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹6,899",https://www.flipkart.com/realme-c30-locked-air...
6,Realme,C30,Denim Black,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹6,899",https://www.flipkart.com/realme-c30-locked-air...
7,REDMI,REDMI 10A,Slate Grey,3 GB,32 GB,Primary Camera,-,16.59 cm (6.53 inch),-,"₹8,436",https://www.flipkart.com/redmi-10a-slate-grey-...
8,vivo,T1 44W,Midnight Galaxy,4 GB,128 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),5000 mAh,"₹14,499",https://www.flipkart.com/vivo-t1-44w-midnight-...
9,Infinix,Smart 6,Light Sea Green,2 GB,64 GB,8 MP + Depth Lens,5MP Front Camera,16.76 cm (6.6 inch),5000 mAh,"₹7,499",https://www.flipkart.com/infinix-smart-6-light...


In [83]:
driver.close()

# Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [84]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)

# Opening the homepage of googlemaps
url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(2)

In [85]:
search = driver.find_element(By.ID,"searchboxinput")                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)

search.send_keys("New Delhi")                                              # entering values in search bar
button = driver.find_element(By.ID,"searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

In [86]:
try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

URL Extracted:  https://www.google.co.in/maps/place/New+Delhi,+Delhi/@28.5272803,77.068897,11z/data=!3m1!4b1!4m5!3m4!1s0x390cfd5b347eb62d:0x52c2b7494e204dce!8m2!3d28.6139391!4d77.2090212
Latitude = 28.5272803, Longitude = 77.068897


In [87]:
search = driver.find_element(By.ID,"searchboxinput")                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)

search.send_keys("Pune")                                              # entering values in search bar
button = driver.find_element(By.ID,"searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

In [88]:
try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

URL Extracted:  https://www.google.co.in/maps/place/Pune,+Maharashtra/@18.5245649,73.7228783,11z/data=!3m1!4b1!4m5!3m4!1s0x3bc2bf2e67461101:0x828d43bf9d9ee343!8m2!3d18.5204303!4d73.8567437
Latitude = 18.5245649, Longitude = 73.7228783


In [89]:
driver.close()

# Q6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in

In [90]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)

# Opening the homepage of googlemaps
url = "https://trak.in/"
driver.get(url)
time.sleep(2)

In [91]:
# click on funding deals
button = driver.find_element(By.XPATH,'//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [107]:
#create empty lists
date=[]
name=[]
industry=[]
subvertical=[]
location=[]
investors_name=[]
investors_type=[]
amount=[]

In [108]:
dates=driver.find_elements(By.XPATH,'//td[@class="column-2"]')
for i in dates[5:29]:
       date.append(i.text)
names=driver.find_elements(By.XPATH,'//td[@class="column-3"]')
for i in names[5:29]:
       name.append(i.text)
industrys=driver.find_elements(By.XPATH,'//td[@class="column-4"]')
for i in industrys[5:29]:
       industry.append(i.text)
subverticals=driver.find_elements(By.XPATH,'//td[@class="column-5"]')
for i in subverticals[5:29]:
       subvertical.append(i.text)
locations=driver.find_elements(By.XPATH,'//td[@class="column-6"]')
for i in locations[5:29]:
       location.append(i.text)
investors_names=driver.find_elements(By.XPATH,'//td[@class="column-7"]')
for i in investors_names[5:29]:
       investors_name.append(i.text)
investors_types=driver.find_elements(By.XPATH,'//td[@class="column-8"]')
for i in investors_types[5:29]:
       investors_type.append(i.text)
amounts=driver.find_elements(By.XPATH,'//td[@class="column-9"]')
for i in amounts[5:29]:
       amount.append(i.text)

In [109]:
print(len(date),len(name),len(industry),len(subvertical),len(location),len(investors_name),len(investors_type),len(amount))

24 24 24 24 24 24 24 24


In [110]:
df=pd.DataFrame({'Date':date,'Name':name,'Industry':industry,'Subvertical':subvertical,'Location':location,'Investors_name':investors_name,'Investors_type':investors_type,'Amount':amount})
df

,Date,Name,Industry,Subvertical,Location,Investors_name,Investors_type,Amount
0,04/03/2021,DealShare,E-commerce,Online shopping platform,"Jaipur, Rajasthan",Innoven Capital,Debt Financing,"250,000,000"
1,31/03/2021,Uniphore,Technology,Conversational Service Automation (CSA),Palo Alto,Sorenson Capital Partners,Series D,"140,000,000"
2,30/03/2021,Dunzo,E-commerce,Hyper-local delivery app,Bengaluru,Krishtal Advisors Pte Ltd,Series E,"8,000,000"
3,30/03/2021,BYJU’S,Edu-tech,Online tutoring,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series F,"460,000,000"
4,23/03/2021,SkilloVilla,Edu-tech,Career and job-oriented upskilling.,Bengaluru,"Titan Capital, others",Seed,"300,000,000"
5,25/03/2021,CityMall,E-commerce,Social ecommerce and online grocery platform,Gurgaon,Accel Partners,Series A,"11,000,000"
6,26/03/2021,DotPe,FinTech,Commerce and payments platform to offline ente...,Gurgaon,PayU,Series A,"27,500,000"
7,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
8,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"
9,19/02/2021,Fingerlix,Hospitality,Semi-cooked food delivery app,Mumbai,"Rhodium Trust, Accel Partners and Swiggy",Series C,"2,747,045.20"


In [111]:
driver.close()

# Q7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [136]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)

# Opening the homepage of digit.in
url = "https://digit.in"
driver.get(url)
time.sleep(2)

In [137]:
top_10=driver.find_element(By.XPATH,'//div[@class="menu"]//ul//li[4]')
top_10.click()

In [138]:
laptops=driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[2]/div[5]/div[1]/div/button[2]")
laptops.click()

In [139]:
best_gaming=driver.find_element(By.XPATH,"//div[@id='laptops']//div[3]//a")
driver.get(best_gaming.get_attribute('href'))

In [140]:
#create an empty list
name = []
Price = []
OS = []
display = []
processor = []
weight = []
dimension = []

In [141]:
#scraping names
names=driver.find_elements(By.XPATH,"//div[@class='right-container']/div/a/h3")
for i in names:
    name.append(i.text)

time.sleep(1)
#scraping operating system
os=driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)

time.sleep(1)
#scraping display
displays=driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in displays:
    display.append(i.text)

time.sleep(1)
#scraping processor
processors=driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in processors:
    processor.append(i.text)


time.sleep(1)
# scraping weight
weights=driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")# extrat weight form xpath
for i in weights:
    weight.append(i.text)

time.sleep(1)
# scraping dimension
dimension=[]
dimensions=driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]") 
for i in dimensions:
    dimension.append(i.text)

time.sleep(1)

#scraping price
price=driver.find_elements(By.XPATH,"//table[@id='summtable']//tr//td[3]")
for i in price:
    Price.append(i.text)

In [145]:
print(len(name),len(Price),len(OS),len(display),len(weight),len(dimension),len(processor))

10 10 10 10 10 10 0


In [146]:
df=pd.DataFrame({'name':name,'Price':Price,'OS':OS,'display':display,'dimension & weight':dimension})
df

,name,Price,OS,display,dimension & weight
0,MSI TITAN GT77-12UHS,N/A,WINDOWS 11 HOME OS,"17.3"" (3840 X 2160) DISPLAY",397 x 330 x 23 mm dimension & 3.3 kg weight
1,ALIENWARE X17 R2,"₹ 389,990",WINDOWS 11 HOME OS,"17.3"" (1920X1080) DISPLAY",399.23 x 299.57 x 14.75 mm dimension & 3.02 kg...
2,ACER PREDATOR TRITON 500 SE PT516-52S,"₹ 300,000",WINDOWS 11 HOME OS,"16"" (2560 X 1600) DISPLAY",358 x 262 x 19.9 mm dimension & 2.4 kg weight
3,OMEN BY HP (16-B1371TX),"₹ 194,000",WINDOWS 11 HOME OS,"16.1"" (2560 X 1440) DISPLAY",369 x 248 x 23 mm dimension & 2.32 kg weight
4,ACER PREDATOR HELIOS 300 AN515-45 (NH.QBRSI.0,"₹ 172,999",WINDOWS 11 HOME OS,"15.6"" (2560 X 1440) DISPLAY",363 x 255 x 23.9 mm dimension & 2.4 kg weight
5,MSI DELTA 15 (A5EFK-083IN),"₹ 188,990",WINDOWS 11 HOME OS,"15.6"" (1920 X 1080) DISPLAY",357 x 247 x 19 mm dimension & 1.9 kg weight
6,OMEN BY HP (16-C0141AX),"₹ 129,899",WINDOWS 11 HOME OS,"16.1"" (2560 X 1440) DISPLAY",36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight
7,LENOVO LEGION 5I PRO (82RF00MGIN),"₹ 230,890",WINDOWS 11 HOME OS,"16"" (2560 X 1600) DISPLAY",359.9 x 264.4 x 19.9 mm dimension & 2.5 kg weight
8,ALIENWARE M15 R5 RYZEN EDITION ICC-C780001WIN,"₹ 144,990",WINDOWS 11 HOME OS,"15.6"" (1920 X 1080) DISPLAY",356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...
9,LENOVO SLIM 7 GEN 6 (82K8002JIN),"₹ 131,990",WINDOWS 11 HOME OS,15.6 MP | NA DISPLAY,356 x 252 x 16 mm dimension & 1.9 kg weight


In [147]:
driver.close()

# Q8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [152]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)

# Opening the homepage of forbes
url = "https://www.forbes.com/?sh=7aaf61ea2254"
driver.get(url)
time.sleep(2)

In [155]:
#clicking the explore button
button = driver.find_element(By.XPATH,"//div[@class='_69hVhdY4']")
button.click()
time.sleep(1)

In [158]:
#select billioners 
billioners = driver.find_element(By.XPATH,"/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[1]/a")
billioners
time.sleep(1)

In [160]:
#select world billioners 
world_billioners= driver.find_element(By.XPATH,"/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[2]/div[2]/ul/li[1]")
world_billioners.click()
time.sleep(1)

In [165]:
#create empty lists
rank=[]
name=[]
worth=[]
age=[]
citizenship=[]
source=[]
industry=[]

In [169]:
ranks=driver.find_elements(By.XPATH,'//div[@class="rank"]')
for i in ranks:
       rank.append(i.text)
        
names=driver.find_elements(By.XPATH,'//div[@class="personName"]')
for i in names:
       name.append(i.text)
        
worths=driver.find_elements(By.XPATH,'//div[@class="netWorth"]')
for i in worths:
       worth.append(i.text)
        
ages= driver.find_elements(By.XPATH,"//div[@class='age']//div")
for i in ages:
        age.append(i.text)
        
citizenships=driver.find_elements(By.XPATH,'//div[@class="countryOfCitizenship"]')
for i in citizenships:
       citizenship.append(i.text)
        
sources=driver.find_elements(By.XPATH,'//span[@class="source-text"]')
for i in sources:
       source.append(i.text)
        
industries= driver.find_elements(By.XPATH,"//div[@class='category']//div")
for i in industries:
        industry.append(i.text)  

In [170]:
print(len(rank),len(name),len(worth),len(age),len(citizenship),len(source),len(industry))

400 400 400 400 400 400 400


In [171]:
df=pd.DataFrame({'Rank':rank,'Name':name,'Worth':worth,'Age':age,'Citizenship':citizenship,'Source':source,'industry':industry})
df

,Rank,Name,Worth,Age,Citizenship,Source,industry
0,1.,Elon Musk,$219 B,,United States,,
1,2.,Jeff Bezos,$171 B,,United States,,
2,3.,Bernard Arnault & family,$158 B,,France,,
3,4.,Bill Gates,$129 B,,United States,,
4,5.,Warren Buffett,$118 B,,United States,,
...,...,...,...,...,...,...,...
395,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
396,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
397,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
398,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


In [172]:
driver.close()

# Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video

In [173]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)

In [174]:
# Opening the homepage of youtube
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(2)

In [179]:
# search any youtube video on search tab
search= driver.find_element(By.XPATH,"/html/body/ytd-app/div[1]/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div[1]/div[1]/input")
search.send_keys('python')
time.sleep(2)

#click on search button
button=driver.find_element(By.XPATH,"/html/body/ytd-app/div[1]/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/button")
button.click()
time.sleep(2)

In [181]:
# click on the video
python_video=driver.find_element(By.XPATH,"/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer[2]/div[1]/div/div[1]/div/h3/a/yt-formatted-string")
python_video.click()

In [182]:
#create empty lists
comments=[]
time_posted=[]
Likes=[]
No_of_Likes=[]

In [184]:
# 1000 time we scroll down by 10000 in order to generate more Comments
for _ in range(500):
    driver.execute_script("window.scrollBy(0,500)")

In [185]:
comment=driver.find_elements(By.ID,"content-text")
for i in comment[0:501]:
       comments.append(i.text)
        
time_postd=driver.find_elements(By.XPATH,'//yt-formatted-string[@class="published-time-text style-scope ytd-comment-renderer"]')
for i in time_postd[0:501]:
       time_posted.append(i.text)
        
like = driver.find_elements(By.XPATH,"//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for i in like:
    Likes.append(i.text)
    
for i in range(1,len(Likes),2)[0:501]:
    No_of_Likes.append(Likes[i])

In [186]:
print(len(comments),len(time_posted),len(No_of_Likes))

501 501 501 2040


In [188]:
df=pd.DataFrame({'Comments':comments,'Time_Posted':time_posted,'No_of_Likes':No_of_Likes})
df

,Comments,Time_Posted,No_of_Likes
0,Please continue python series from beginner to...,8 months ago,2.9K
1,"Shraddha Di, please make a well structured cou...",8 months ago,1K
2,I can say after 4 years of confusion across bo...,3 months ago (edited),119
3,"very well explained content, I an from non IT ...",2 weeks ago,3
4,"Hi Apna college.. Amazing content, material, a...",7 months ago (edited),44
...,...,...,...
496,Very clear and understanding voice,2 months ago,
497,You are really a great teacher Machine learni...,1 month ago,
498,very nicely delivered,11 days ago,
499,And didi very very thanks for this the explana...,3 months ago,1


In [189]:
driver.close()

# Q10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description

In [20]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)

# Opening the homepage of youtube
url = "https://www.hostelworld.com/"
driver.get(url)
time.sleep(2)

In [22]:
#search the required location
london_tag=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/div/input')
london_tag.send_keys('London')

In [23]:
#Click on the london button
london_btn=driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div")
london_btn.click() 

In [24]:
#click on lets go button
go_btn=driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[5]/button")
go_btn.click() 

In [25]:
#create empty lists
Hostel_Name = []
Distance = []
overall_review = []
total_reviews = []
facilities = []
price = []
Rating = []
property_description = []

In [26]:
while(True):
    
    # Hostel name
    names = driver.find_elements(By.XPATH,'//h2[@class="title title-6"]')
    for name in names:
        Hostel_Name.append(name.text)
    time.sleep(2)
        
    # Distance from city centre
    dis = driver.find_elements(By.XPATH,'//span[@class="description"]')
    for d in dis:
        Distance.append(d.text)
    time.sleep(2)
        
    # Overall Review    
    review = driver.find_elements(By.XPATH,'//div[@class="keyword"]//span')
    for r in review:
        overall_review.append(r.text)
    time.sleep(2)
    
    # Total No of reviews     
    t_review = driver.find_elements(By.XPATH,'//div[@class="reviews"]')
    for tr in t_review:
        total_reviews.append(tr.text)
    time.sleep(2)
    
    # facilities
    service = driver.find_elements(By.XPATH,'//div[@class="facilities-label facilities"]')
    for s in service:
        facilities.append(s.text)
    time.sleep(2)
    
    # Prices    
    prices = driver.find_elements(By.XPATH,'//div[@class="price-col"]')
    for p in prices:
        price.append(p.text)
    time.sleep(2)    
        
    try:
        next_button = driver.find_element(By.XPATH,'//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break
        
time.sleep(4)        
        
# Separate  Privates_From price  and  Dorms_From price
Privates_From = []
for i in range(0,len(price),2):
    Privates_From.append(price[i])
time.sleep(2)

Dorms_From = []
for i in range(1,len(price),2):
    Dorms_From.append(price[i])

In [28]:
# scrape Hostels URL
hostel_url = []

while(True):
    urls = driver.find_elements(By.XPATH,'//h2[@class="title title-6"]//a')
    for url in urls:
        hostel_url.append(url.get_attribute("href"))
    time.sleep(2)    
        
    try:
        next_button = driver.find_element(By.XPATH,'//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break

In [29]:
Rate = []
for page in hostel_url:
    driver.get(page)
    
    # Rating
    try:
        ratings = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[1]/section/div[6]/div/div[1]/div[1]/div[1]')
        Rate.append(ratings.text)
    except NoSuchElementException:
        Rate.append("No Rating")  
    time.sleep(2)
    
    
    # Property Description
    try:
        pd = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[1]/section/div[6]/div/div[2]/div[2]/div/div[2]')
        property_description.append(pd.text)
    except NoSuchElementException:
        property_description.append("No Description")  

    
time.sleep(2)        
# remove extra data from Rating     
all_text = []
for i in Rate:
    all_text.append(i.split())
time.sleep(2)

for i in all_text:
    Rating.append(i[0]) 

In [31]:
print(len(Hostel_Name),len(Distance),len(total_reviews),len(facilities),len(Privates_From),len(Dorms_From))

30 30 30 30 30 30


In [32]:
import pandas as pd
df=pd.DataFrame({'Name':Hostel_Name,'Distance_city':Distance,'Tot_review':total_reviews,'Pri_frm_price':Privates_From,'Dorms_frm_price':Dorms_From,'Facility':facilities})
df

,Name,Distance_city,Tot_review,Pri_frm_price,Dorms_frm_price,Facility
0,Selina Camden,Hostel - 5.5km from city centre,36 Total Reviews,Privates From\nRs22100.48 Rs18785,No Dorms Available,Free WiFi
1,Palmers Lodge - Swiss Cottage,Hostel - 6.5km from city centre,15565 Total Reviews,No Privates Available,Dorms From\nRs2055,Free WiFi\nFollows Covid-19 sanitation guidance
2,St Christopher's Village,Hostel - 1.8km from city centre,11479 Total Reviews,Privates From\nRs12300.56 Rs11071,Dorms From\nRs2238.7 Rs2015,Free WiFi\nFollows Covid-19 sanitation guidance
3,Wombat's City Hostel London,Hostel - 3.6km from city centre,14009 Total Reviews,Privates From\nRs17312.05 Rs14369,Dorms From\nRs4058.71 Rs3044,Free WiFi\nFollows Covid-19 sanitation guidance
4,Generator London,Hostel - 3km from city centre,7138 Total Reviews,Privates From\nRs9089,Dorms From\nRs2604,Free WiFi\nFollows Covid-19 sanitation guidance
5,Astor Museum Inn,Hostel - 2.2km from city centre,9046 Total Reviews,Privates From\nRs10129,Dorms From\nRs2763,Free WiFi\nFollows Covid-19 sanitation guidance
6,Urbany Hostel London,Hostel - 5.4km from city centre,453 Total Reviews,Privates From\nRs10940,Dorms From\nRs2006,Free WiFi\nFollows Covid-19 sanitation guidance
7,Astor Hyde Park,Hostel - 4.3km from city centre,11666 Total Reviews,No Privates Available,Dorms From\nRs2210,Free WiFi\nFollows Covid-19 sanitation guidance
8,Safestay London Elephant & Castle,Hostel - 1.7km from city centre,4474 Total Reviews,Privates From\nRs14918,Dorms From\nRs3895,Free WiFi\nFollows Covid-19 sanitation guidance
9,Smart Camden Inn Hostel,Hostel - 4.4km from city centre,2920 Total Reviews,No Privates Available,Dorms From\nRs2019,Free WiFi\nFollows Covid-19 sanitation guidance


In [33]:
driver.close()